# Example: Improving Math Reasoning with a Custom Recipe for Automated Prompt Engineering (DSPy)


In [1]:
import asyncio
import json
import os
import random
from typing import Dict, Optional

import altair as alt
import numpy as np
import pandas as pd
from dataset.dataset import get_examples, is_correct
from scipy.stats import norm
from tensorzero import AsyncTensorZeroGateway, InferenceResponse
from tqdm.asyncio import tqdm_asyncio

## Setup TensorZero Client

In [2]:
t0 = await AsyncTensorZeroGateway.build_http(
    gateway_url="http://localhost:3000", timeout=30
)

# 

## Setup Dataset

In [3]:
train_examples = get_examples("train")
random.shuffle(train_examples)

test_examples = get_examples("test")
random.shuffle(test_examples)

print(train_examples[0])

7473 train examples
1319 test examples
{'question': 'Lana and her friends go to a theater on the weekend to watch a new play Joan told them was live. When they arrived, Lana saw 100 cars in the front parking lot. She saw two times more vehicles at the back than there were in the front parking lot. If the total number of cars at the end of the play was 700, how many more cars packed into the parking lot during the play?\n', 'answer': 'The number of cars in the back parking lot was 100*2 = <<100*2=200>>200 times more than those in the front parking lot.\nThere were 200+100 = <<200+100=300>>300 cars in the back parking lot.\nA total of 300+100 = <<300+100=400>>400 cars were present before the play began\nThe total number of cars that parked in the theater during the play is 700-400 = <<700-400=300>>300 cars\n#### 300<|endoftext|>'}


## Helper Functions

In [4]:
async def solve_math_problem(
    question: str,
    *,
    variant_name: Optional[str] = None,
    dryrun: bool = False,
) -> Optional[InferenceResponse]:
    try:
        return await t0.inference(
            function_name="solve_math_problem",
            input={"messages": [{"role": "user", "content": {"question": question}}]},
            cache_options={"enabled": "on"},
            variant_name=variant_name,
            dryrun=dryrun,
        )
    except Exception as e:
        print(f"Error ({type(e)}): {e}")
        return None

In the function below, the only feedback provided to TensorZero is whether the output of the function is correct.
We do not provide the correct answer in cases of mistakes.


In [5]:
async def solve_and_score_math_problem(
    example: Dict[str, str],
    *,
    variant_name: Optional[str] = None,
    dryrun: bool = False,
) -> Optional[bool]:
    # Run the TensorZero function on the example
    response = await solve_math_problem(
        example["question"], variant_name=variant_name, dryrun=dryrun
    )

    # Inference failed completely, disregard this example
    if response is None:
        return None

    # Inference succeeded, but the first block is not text, so we consider it incorrect
    first_block = response.content[0]
    if first_block.type != "text":
        return False

    # Inference succeeded, and the first block is text, so we score the example
    correct = is_correct(first_block.text, example)

    # Store the feedback in the database
    # (skip if dryrun since we're not storing the inferences, which will trigger an error)
    if not dryrun:
        await t0.feedback(
            metric_name="correct",
            value=correct,
            inference_id=response.inference_id,
            dryrun=dryrun,
        )

    return correct

## Run Examples

Run the TensorZero function on the training examples, grade the answers, and store the feedback in the database.


In [6]:
# Reduce the number of concurrent inferences to avoid rate limiting
MAX_CONCURRENT_INFERENCES = 100

semaphore = asyncio.Semaphore(MAX_CONCURRENT_INFERENCES)


async def run_inference(
    example: Dict[str, str], *, variant_name: Optional[str] = None, dryrun: bool = False
) -> Optional[bool]:
    async with semaphore:
        return await solve_and_score_math_problem(
            example, variant_name=variant_name, dryrun=dryrun
        )

In [7]:
NUM_TRAINING_INFERENCES = 1000

coroutines = [
    run_inference(example) for example in train_examples[:NUM_TRAINING_INFERENCES]
]

results = await tqdm_asyncio.gather(*coroutines, desc="Running training inferences")

Running training inferences:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-08-12T09:34:21.633508Z  WARN tensorzero::client_input: Deprecation Warning: passing in an object for `content` is deprecated. Please use an array of content blocks instead.
2025-08-12T09:34:21.633939Z  WARN tensorzero::client_input: Deprecation Warning: passing in an object for `content` is deprecated. Please use an array of content blocks instead.
2025-08-12T09:34:21.634095Z  WARN tensorzero::client_input: Deprecation Warning: passing in an object for `content` is deprecated. Please use an array of content blocks instead.
2025-08-12T09:34:21.634367Z  WARN tensorzero::client_input: Deprecation Warning: passing in an object for `content` is deprecated. Please use an array of content blocks instead.
2025-08-12T09:34:21.634572Z  WARN tensorzero::client_input: Deprecation Warning: passing in an object for `content` is deprecated. Please use an array of content blocks instead.
2025-08-12T09:34:21.634704Z  WARN tensorzero::client_input: Deprecation Warning: passing in an object for `con

Running training inferences:   1%|▏         | 14/1000 [00:02<01:59,  8.25it/s]

2025-08-12T09:34:23.808950Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
Error (<class 'tensorzero.types.TensorZeroError'>): TensorZeroError (status code 502): {"error":"All variants failed with errors: gpt_4o_mini_baseline: All model providers failed to infer with errors: openai: Error 429 Too Many Requests from openai client: {\n    \"error\": {\n        \"message\": \"You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.\",\n        \"type\": \"insufficient_quota\",\n        \"param\": null,\n        \"code\": \"insufficient_quota\"\n    }\n}\n\ngpt_4o_mini_optimized: All model providers failed to infer with errors: openai: Error 429 Too Many Requests from openai client: {\n    \"error\": {\n        \"message\": \"You exceeded your current quota, please check

Running training inferences:   2%|▏         | 22/1000 [00:02<01:18, 12.48it/s]

...

2025-08-12T09:34:52.584811Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
Error (<class 'tensorzero.types.TensorZeroError'>): TensorZeroError (status code 502): {"error":"All variants failed with errors: gpt_4o_mini_baseline: All model providers failed to infer with errors: openai: Error 429 Too Many Requests from openai client: {\n    \"error\": {\n        \"message\": \"You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.\",\n        \"type\": \"insufficient_quota\",\n        \"param\": null,\n        \"code\": \"insufficient_quota\"\n    }\n}\n\ngpt_4o_mini_optimized: All model providers failed to infer with errors: openai: Error 429 Too Many Requests from openai client: {\n    \"error\": {\n        \"message\": \"You exceeded your current quota, please check

Running training inferences:  97%|█████████▋| 970/1000 [00:31<00:00, 49.25it/s]

2025-08-12T09:34:52.786944Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
2025-08-12T09:34:52.786952Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
2025-08-12T09:34:52.786983Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
2025-08-12T09:34:52.787724Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
2025-08-12T09:34:52.787767Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
Error (<class 'tensorzero.types.TensorZeroError'>): TensorZeroError (status code 502): {"error":"All variants failed with errors: gpt_4o_mini_baseline: All model providers failed to inf

Running training inferences: 100%|██████████| 1000/1000 [00:31<00:00, 31.71it/s]

2025-08-12T09:34:53.031715Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
Error (<class 'tensorzero.types.TensorZeroError'>): TensorZeroError (status code 502): {"error":"All variants failed with errors: gpt_4o_mini_baseline: All model providers failed to infer with errors: openai: Error 429 Too Many Requests from openai client: {\n    \"error\": {\n        \"message\": \"You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.\",\n        \"type\": \"insufficient_quota\",\n        \"param\": null,\n        \"code\": \"insufficient_quota\"\n    }\n}\n\ngpt_4o_mini_optimized: All model providers failed to infer with errors: openai: Error 429 Too Many Requests from openai client: {\n    \"error\": {\n        \"message\": \"You exceeded your current quota, please check

In the cell below, we evaluate the accuracy of a variant on some of the test examples. If you generate a new variant, you should run this cell with the new variant name to evaluate it.


In [8]:
NUM_TEST_INFERENCES = 200

VARIANTS = [
    "gpt_4o_mini_baseline",
    # The provided optimized variant was generated by DSPy using the code below
    # "gpt_4o_mini_optimized",
]

accuracies = {}

for variant_name in VARIANTS:
    # We use dryrun=True here to avoid leaking the test set into the database
    coroutines = [
        run_inference(example, variant_name=variant_name, dryrun=True)
        for example in test_examples[:NUM_TEST_INFERENCES]
    ]

    results = await tqdm_asyncio.gather(*coroutines, desc="Running test inferences")

    # Filter out None values from results
    total_results = len(results)
    results = [result for result in results if result is not None]
    success_rate = len(results) / total_results
    print(f"Success rate: {success_rate:.1%}")

    accuracy = sum(results) / len(results)
    n = len(results)
    z = norm.ppf(0.975)  # 95% confidence interval
    margin_of_error = z * np.sqrt((accuracy * (1 - accuracy)) / n)

    print(f"Accuracy: {accuracy:.4f}")
    print(
        f"95% Confidence Interval: ({accuracy - margin_of_error:.4f}, {accuracy + margin_of_error:.4f})"
    )

    accuracies[variant_name] = accuracy

Running test inferences:   0%|          | 0/200 [00:00<?, ?it/s]

2025-08-12T09:34:53.177750Z  WARN tensorzero::client_input: Deprecation Warning: passing in an object for `content` is deprecated. Please use an array of content blocks instead.
2025-08-12T09:34:53.177905Z  WARN tensorzero::client_input: Deprecation Warning: passing in an object for `content` is deprecated. Please use an array of content blocks instead.
2025-08-12T09:34:53.178155Z  WARN tensorzero::client_input: Deprecation Warning: passing in an object for `content` is deprecated. Please use an array of content blocks instead.
2025-08-12T09:34:53.178322Z  WARN tensorzero::client_input: Deprecation Warning: passing in an object for `content` is deprecated. Please use an array of content blocks instead.
2025-08-12T09:34:53.178429Z  WARN tensorzero::client_input: Deprecation Warning: passing in an object for `content` is deprecated. Please use an array of content blocks instead.
2025-08-12T09:34:53.178536Z  WARN tensorzero::client_input: Deprecation Warning: passing in an object for `con

Running test inferences:  11%|█         | 22/200 [00:00<00:03, 50.27it/s]

2025-08-12T09:34:53.631568Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
Error (<class 'tensorzero.types.TensorZeroError'>): TensorZeroError (status code 502): {"error":"All variants failed with errors: gpt_4o_mini_baseline: All model providers failed to infer with errors: openai: Error 429 Too Many Requests from openai client: {\n    \"error\": {\n        \"message\": \"You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.\",\n        \"type\": \"insufficient_quota\",\n        \"param\": null,\n        \"code\": \"insufficient_quota\"\n    }\n}\n"}
Error (<class 'tensorzero.types.TensorZeroError'>): TensorZeroError (status code 502): {"error":"All variants failed with errors: gpt_4o_mini_baseline: All model providers failed to infer with errors: openai: Error 4

Running test inferences:  22%|██▏       | 43/200 [00:00<00:02, 55.12it/s]

...

Running test inferences:  78%|███████▊  | 157/200 [00:01<00:00, 109.52it/s]

2025-08-12T09:34:54.756418Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
2025-08-12T09:34:54.756943Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
2025-08-12T09:34:54.756953Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
2025-08-12T09:34:54.756948Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
2025-08-12T09:34:54.756953Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
2025-08-12T09:34:54.756943Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
Error (<class 'tensorz

Running test inferences: 100%|██████████| 200/200 [00:09<00:00, 20.71it/s] 

2025-08-12T09:35:02.816464Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
2025-08-12T09:35:02.816497Z  WARN tensorzero_internal::error: Request failed: HTTP status server error (502 Bad Gateway) for url (http://localhost:3000/inference)
Error (<class 'tensorzero.types.TensorZeroError'>): TensorZeroError (status code 502): {"error":"All variants failed with errors: gpt_4o_mini_baseline: All model providers failed to infer with errors: openai: Error 429 Too Many Requests from openai client: {\n    \"error\": {\n        \"message\": \"You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.\",\n        \"type\": \"insufficient_quota\",\n        \"param\": null,\n        \"code\": \"insufficient_quota\"\n    }\n}\n"}
Error (<class 'tensorzero.types.TensorZeroError'>): T

ZeroDivisionError: division by zero

## Plot Results

In [ ]:
scores_df = pd.DataFrame(
    [
        {"Variant": variant_name, "Score": accuracy}
        for variant_name, accuracy in accuracies.items()
    ]
)

scores_df

In [ ]:
chart = (
    alt.Chart(scores_df)
    .encode(
        x=alt.X("Score:Q", axis=alt.Axis(format="%"), scale=alt.Scale(domain=[0, 1])),
        y="Variant:N",
        text=alt.Text("Score:Q", format=".1%"),
    )
    .properties(title="Metrics by Variant")
)

chart = chart.mark_bar() + chart.mark_text(align="left", dx=2)

chart

At this point, we could run any TensorZero recipe to generate a new variant which might perform better using this historical data. You can go try this!

Below, we include an example of how to use an external library, [DSPy](https://dspy-docs.vercel.app/), to automatically optimize a prompt for this function.
Given that the ClickHouse database TensorZero uses easily allows for the querying of historical inference and feedback data into Pandas DataFrames, it is easy to integrate with nearly any ML library yourself.


## Automated Prompt Engineering with DSPy

The rest of this notebook shows how we can pull data from the TensorZero data model in ClickHouse and use it to optimize a prompt for a function using DSPy.
Given that there are many strategies for prompt optimization in DSPy, we can use the same code skeleton to try a lot of different strategies.
However, there are a few things (table name, feedback name, chat function type, etc) that we have set specifically for this example.
You can change them to fit your needs.
At a high level the notebook below does the following:

1. Pull data from ClickHouse and convert it into a DSPy dataset.
2. Run a prompt optimization loop using one of the teleprompting classes supported by DSPy.
3. Print the optimized prompt from the history so that you can write it to a minijinja file.

**Note:** DSPy does not model the chat completion interface commonly used by language models. So, we only support functions that have inputs into the user prompt, that only use text output, that are single-turn functions, and that have a flat JSON schema for input, i.e. functions that take a list of primitive types as input into the user schema and output text or a flat JSON object.

To get started:

- Set the `TENSORZERO_CLICKHOUSE_URL` environment variable. 
- Set the `OPENAI_API_KEY` environment variable.
- Update the following parameters to those that apply to your use case.


In [ ]:
import dspy
from clickhouse_connect import get_client
from dspy.datasets import Dataset

You can swap the client below for any of the ones supported [here](https://dspy.ai/learn/programming/language_models/) in case you want DSPy to use a different language model.


In [ ]:
lm_client = dspy.OpenAI(model="gpt-4o-mini")
dspy.configure(lm=lm_client)

In [ ]:
# A simple function signature for the `solve_math_problem` function
function_signature = "input -> output"

Initialize the ClickHouse client.


In [ ]:
assert "TENSORZERO_CLICKHOUSE_URL" in os.environ, (
    "TENSORZERO_CLICKHOUSE_URL environment variable not set"
)
clickhouse_client = get_client(dsn=os.environ["TENSORZERO_CLICKHOUSE_URL"])

Grab the dataset of examples which were successful according to the metric.


In [ ]:
query = """
SELECT 
    i.variant_name, 
    i.input, 
    i.output, 
    i.episode_id,
    f.value
FROM 
    ChatInference i
JOIN 
    (SELECT
        target_id,
        value,
        ROW_NUMBER() OVER (PARTITION BY target_id ORDER BY timestamp DESC) as rn
    FROM BooleanMetricFeedback
    WHERE
        metric_name = 'correct'
        AND value = true
    ) f ON i.id = f.target_id and f.rn = 1
WHERE 
    i.function_name = 'solve_math_problem'
LIMIT %(max_samples)s
"""

params = {
    "max_samples": 1000,
}

df = clickhouse_client.query_df(query, params)

df.head()

In [ ]:
def parse_dspy_compatible_inputs(input_raw: str) -> Optional[Dict[str, str]]:
    """
    Checks that the input of this Inference is in the correct format for DSPy.
    Then returns the dictionary of inputs.
    """
    try:
        parsed_input = json.loads(input_raw)
    except json.JSONDecodeError:
        print(f"Input is not valid JSON: {input_raw}")
        return None
    messages = parsed_input.get("messages", None)
    if messages is None:
        print(f"Input contains no messages: {input_raw}")
        return None
    if len(messages) != 1:
        print(f"Input contains more than one message: {input_raw}")
        return None
    message = messages[0]
    content = message.get("content", None)
    if content is None:
        print(f"Input contains no content: {input_raw}")
        return None
    if len(content) != 1:
        print(f"Input must contain exactly one content item: {input_raw}")
        return None
    content = content[0]
    if content["type"] != "text":
        print(f"Input contains non-text content: {input_raw}")
        return None
    value = content.get("value", None)
    if value is None:
        print(f"Input contains no value: {input_raw}")
        return None
    return value

In [ ]:
# Parse the input column into a list of dicts and create a new DataFrame with parsed content
parsed_inputs = df["input"].apply(parse_dspy_compatible_inputs)

In [ ]:
def parse_outputs(output_raw: str) -> Optional[str]:
    try:
        parsed_output = json.loads(output_raw)
    except json.JSONDecodeError:
        print(f"Output is not valid JSON: {output_raw}")
        return None
    if len(parsed_output) != 1:
        print(f"Output contains more than one message: {output_raw}")
        return None
    message = parsed_output[0]
    if message["type"] != "text":
        print(f"Output contains non-text content: {output_raw}")
        return None
    value = message.get("text", None)
    if value is None:
        print(f"Output contains no value: {output_raw}")
        return None
    return value

In [ ]:
# Parse the output column and create a new DataFrame with parsed content
parsed_outputs = df["output"].apply(parse_outputs)

In [ ]:
all_data = pd.concat([parsed_inputs, parsed_outputs], axis=1)
all_data = all_data.dropna()

In [ ]:
all_data.head()

In [ ]:
class TensorZeroDSPyDataset(Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        dev_fraction: float = 0.1,
    ):
        # Randomly shuffle the DataFrame
        df = df.sample(frac=1, random_state=42).reset_index(drop=True)

        # Extract the 'question' string from the 'input' column
        df["input"] = df["input"].apply(lambda x: x["question"])

        # Calculate the number of samples for train and dev sets
        total_samples = len(df)
        dev_samples = int(total_samples * dev_fraction)
        train_samples = total_samples - dev_samples

        # Split the DataFrame
        train_df = df.iloc[:train_samples]
        dev_df = df.iloc[train_samples:]

        # Split the DataFrame
        self._train = train_df.to_dict(orient="records")
        self._dev = dev_df.to_dict(orient="records")
        self._test = None
        self.train_size = len(self._train)
        self.dev_size = len(self._dev)
        super().__init__(
            train_size=self.train_size,
            dev_size=self.dev_size,
            test_size=0,
        )

        print(f"Train set: {len(self._train)} samples")
        print(f"Dev set: {len(self._dev)} samples")

In [ ]:
dataset = TensorZeroDSPyDataset(all_data)

In [ ]:
dspy_function = dspy.Predict(function_signature)


class Predictor(dspy.Module):
    def __init__(self, signature: dspy.Signature):
        super().__init__()
        self.prog = dspy.Predict(signature)

    def forward(self, **inputs):
        return self.prog(**inputs)

You can swap the teleprompter with any of the teleprompting classes supported by DSPy [here](https://dspy-docs.vercel.app/docs/building-blocks/optimizers).


In [ ]:
from dspy.teleprompt import LabeledFewShot

teleprompter = LabeledFewShot(k=5)
optimized_function = teleprompter.compile(
    Predictor(function_signature), trainset=dataset.train
)

In [ ]:
print(dataset.dev[0]["input"])

We run an example inference to get the prompt from the history.


In [ ]:
optimized_function(input="test_input")

Let's parse out the prompt from the history.


In [ ]:
dspy_prompt = lm_client.history[-1]["prompt"]
# we parse the actual inference input out of the prompt (DSPy does not separate the prompt from the inputs in this history)
dspy_prompt = "---".join(dspy_prompt.split("---")[:-1])

# DSPy does not know the output format for GSM8k, so we add it manually
merged_prompt = f"""
You are tasked with solving a math problem. You will be given an open-ended question that should require arithmetic to solve.

Feel free to work through the problem step-by-step in your response, but once you have found the solution, please complete your response with:
#### your_answer


---

{dspy_prompt}

---

REMEMBER: End your response with `#### your_answer`, where `your_answer` should be an integer with no other punctuation.
""".strip()

print(merged_prompt)

Write the optimized user prompt to a minijinja file and try it out! You can skip to the training cell and use the new variant name to evaluate.
